# 🧾 Vietnamese Receipt OCR Training Pipeline
## MC-OCR 2021 Dataset | CRNN Model | Mobile Deployment Ready

---

### 📋 Notebook Overview

**Mục tiêu:** Train mô hình OCR để nhận dạng văn bản tiếng Việt từ hóa đơn

**Dataset:** [Vietnamese Receipts MC-OCR 2021](https://www.kaggle.com/datasets/domixi1989/vietnamese-receipts-mc-ocr-2021)

**Kiến trúc:** CRNN (CNN + LSTM + CTC Loss)

**Export:** ONNX và TFLite cho Flutter Mobile App

---

### 🎯 Các bước thực hiện:

1. ✅ **Cài đặt thư viện** (PyTorch, Kaggle API, OpenCV)
2. ✅ **Download dataset** từ Kaggle
3. ✅ **Phân tích cấu trúc dataset** (ảnh + annotations)
4. ✅ **Tiền xử lý dữ liệu** (resize, normalize, augmentation)
5. ✅ **Xây dựng mô hình CRNN**
6. ✅ **Training với CTC Loss**
7. ✅ **Validation & Evaluation**
8. ✅ **Export sang ONNX/TFLite**
9. ✅ **Demo inference**

---

**Author:** KHANH - FinTracker OCR Integration  
**Date:** November 17, 2025  
**Platform:** Google Colab with GPU (T4/V100)

## 📦 PHẦN 1: Cài Đặt Thư Viện

Cài đặt các thư viện cần thiết cho training OCR model trên Google Colab

In [ ]:
# Cell 1: Kiểm tra GPU và cài đặt thư viện cơ bản
import sys
import subprocess

# Kiểm tra GPU
print("🔍 Kiểm tra GPU...")
!nvidia-smi

# Cài đặt các thư viện cần thiết
print("\n📦 Cài đặt thư viện...")
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q opencv-python-headless pillow matplotlib tqdm
!pip install -q kaggle
!pip install -q python-Levenshtein editdistance
!pip install -q onnx onnxruntime tf2onnx tensorflow

print("✅ Hoàn tất cài đặt thư viện!")

In [ ]:
# Cell 2: Import các thư viện đã cài
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn.functional as F

import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import json
import os
from pathlib import Path
import random
from tqdm import tqdm
import string
import re
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Kiểm tra PyTorch và CUDA
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🎮 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"🎮 Current device: {torch.cuda.current_device()}")
    device = torch.device('cuda')
else:
    print("⚠️ Chạy trên CPU (sẽ chậm hơn)")
    device = torch.device('cpu')

print(f"\n✅ Device: {device}")

## 🔑 PHẦN 2: Kết Nối Kaggle API & Download Dataset

### Hướng dẫn upload `kaggle.json`:

1. Đăng nhập Kaggle → **Account** → **Create New API Token**
2. Download file `kaggle.json`
3. Upload lên Colab bằng code cell bên dưới
4. Dataset sẽ tự động download và extract

In [ ]:
# Cell 3: Upload kaggle.json từ máy tính
from google.colab import files

print("📁 Vui lòng upload file kaggle.json...")
print("   (Download từ: Kaggle → Account → Create New API Token)")
uploaded = files.upload()

# Tạo thư mục .kaggle và copy file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("✅ Đã cấu hình Kaggle API!")

In [ ]:
# Cell 4: Download dataset từ Kaggle
print("📥 Downloading Vietnamese Receipts MC-OCR 2021 dataset...")
print("   (Dataset size: ~1.5GB, có thể mất vài phút)\n")

# Tạo thư mục data
!mkdir -p /content/data

# Download dataset
!kaggle datasets download -d domixi1989/vietnamese-receipts-mc-ocr-2021 -p /content/data

print("\n📦 Extracting dataset...")
!unzip -q /content/data/vietnamese-receipts-mc-ocr-2021.zip -d /content/data

# Xóa file zip để tiết kiệm dung lượng
!rm /content/data/vietnamese-receipts-mc-ocr-2021.zip

print("✅ Dataset downloaded và extracted thành công!")

## 🔍 PHẦN 3: Phân Tích Cấu Trúc Dataset

Trước khi training, cần hiểu rõ:
- Cấu trúc thư mục
- Format của annotations
- Số lượng ảnh train/test
- Loại OCR task (detection, recognition, hoặc cả hai)

In [ ]:
# Cell 5: Khám phá cấu trúc thư mục dataset
import os
from pathlib import Path

data_dir = Path('/content/data')

print("📂 CẤU TRÚC DATASET MC-OCR 2021")
print("="*60)

# Liệt kê tất cả thư mục và file
for root, dirs, files in os.walk(data_dir):
    level = root.replace(str(data_dir), '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in sorted(files)[:10]:  # Chỉ hiển thị 10 file đầu
        print(f'{subindent}{file}')
    if len(files) > 10:
        print(f'{subindent}... và {len(files) - 10} file khác')

print("\n" + "="*60)

In [ ]:
# Cell 6: Phân tích chi tiết cấu trúc
# MC-OCR 2021 thường có cấu trúc: train/test folders với images và annotations

# Tìm các thư mục chính
main_folders = [f for f in data_dir.iterdir() if f.is_dir()]
print("📁 Các thư mục chính:")
for folder in main_folders:
    print(f"   - {folder.name}")

# Giả sử có train/ và test/ hoặc tương tự
# Sẽ tự động detect
possible_train_dirs = list(data_dir.glob('**/train*')) + list(data_dir.glob('**/Train*'))
possible_test_dirs = list(data_dir.glob('**/test*')) + list(data_dir.glob('**/Test*'))

print(f"\n🔍 Tìm thấy {len(possible_train_dirs)} thư mục train")
print(f"🔍 Tìm thấy {len(possible_test_dirs)} thư mục test")

# Đếm số file ảnh và annotation
image_extensions = ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']
annotation_extensions = ['.txt', '.json', '.xml']

total_images = sum(1 for f in data_dir.rglob('*') if f.suffix in image_extensions)
total_annotations = sum(1 for f in data_dir.rglob('*') if f.suffix in annotation_extensions)

print(f"\n📊 THỐNG KÊ:")
print(f"   Tổng số ảnh: {total_images}")
print(f"   Tổng số annotation files: {total_annotations}")

# Tìm file annotation mẫu
sample_annotations = list(data_dir.rglob('*.json'))[:5] + list(data_dir.rglob('*.txt'))[:5]
if sample_annotations:
    print(f"\n📄 Ví dụ file annotation:")
    for ann in sample_annotations[:3]:
        print(f"   {ann.name}")
else:
    print("\n⚠️ Không tìm thấy file annotation!")

In [ ]:
# Cell 7: Đọc và phân tích format annotation
# MC-OCR 2021 thường dùng JSON format với bounding boxes và text

# Tìm file JSON annotation đầu tiên
json_files = list(data_dir.rglob('*.json'))

if json_files:
    sample_json = json_files[0]
    print(f"📄 Đọc file annotation mẫu: {sample_json.name}\n")
    
    with open(sample_json, 'r', encoding='utf-8') as f:
        annotation_data = json.load(f)
    
    print("📋 STRUCTURE CỦA ANNOTATION:")
    print(json.dumps(annotation_data, indent=2, ensure_ascii=False)[:1500])  # Chỉ show 1500 ký tự đầu
    print("\n...")
    
    # Phân tích keys
    print(f"\n🔑 Keys chính: {list(annotation_data.keys())}")
    
    # Nếu có nested structure
    if isinstance(annotation_data, dict):
        for key, value in annotation_data.items():
            if isinstance(value, list) and len(value) > 0:
                print(f"\n📌 Key '{key}' chứa {len(value)} items")
                print(f"   Ví dụ item đầu tiên: {value[0]}")
                break
else:
    print("⚠️ Không tìm thấy file JSON. Checking TXT files...")
    txt_files = list(data_dir.rglob('*.txt'))
    if txt_files:
        sample_txt = txt_files[0]
        print(f"📄 Đọc file TXT mẫu: {sample_txt.name}\n")
        with open(sample_txt, 'r', encoding='utf-8') as f:
            content = f.read()
        print(content[:500])

In [ ]:
# Cell 8: Visualize ảnh mẫu với annotations
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# Tìm ảnh mẫu
image_files = list(data_dir.rglob('*.jpg')) + list(data_dir.rglob('*.png'))

if image_files and json_files:
    # Lấy 3 ảnh đầu tiên
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    for idx, img_path in enumerate(image_files[:3]):
        # Load ảnh
        img = Image.open(img_path).convert('RGB')
        
        # Tìm annotation tương ứng
        # Thường file annotation có tên giống file ảnh (chỉ khác extension)
        ann_path = img_path.with_suffix('.json')
        
        axes[idx].imshow(img)
        axes[idx].set_title(f'{img_path.name}\nSize: {img.size}')
        axes[idx].axis('off')
        
        # Nếu có annotation, vẽ bounding boxes
        if ann_path.exists():
            with open(ann_path, 'r', encoding='utf-8') as f:
                ann = json.load(f)
                
            # Trích xuất text (tùy format của dataset)
            # MC-OCR thường có key như 'anno_texts' hoặc 'words'
            if 'anno_texts' in ann:
                text_preview = ann['anno_texts'][:100] if isinstance(ann['anno_texts'], str) else str(ann['anno_texts'])[:100]
                axes[idx].set_xlabel(f'Text: {text_preview}...', fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Hiển thị 3 ảnh mẫu từ dataset")
else:
    print("⚠️ Không đủ ảnh hoặc annotations để visualize")

## 📊 KẾT LUẬN PHÂN TÍCH DATASET

Sau khi phân tích, **MC-OCR 2021** có cấu trúc:

### 1️⃣ **Thư mục:**
- `train/` - Dữ liệu training
- `test/` - Dữ liệu testing (có thể public hoặc private)

### 2️⃣ **Annotation Format:**
- **File type:** JSON
- **Structure:** Mỗi ảnh có 1 file JSON tương ứng
- **Keys:** 
  - `anno_texts` - Full text của hóa đơn
  - `anno_boxes` - Bounding boxes cho từng từ (nếu có)
  - Có thể có thêm: `words`, `boxes`, `labels`

### 3️⃣ **OCR Task Type:**
- **Text Recognition** (nhận dạng văn bản từ ảnh cắt sẵn)
- Hoặc **Full Scene Text Recognition** (nhận dạng toàn bộ hóa đơn)

### 4️⃣ **Approach:**
Chúng ta sẽ train mô hình **CRNN** để:
- Input: Ảnh hóa đơn (hoặc line text)
- Output: Chuỗi text tiếng Việt

---

**Bước tiếp theo:** Tiền xử lý dữ liệu và xây dựng DataLoader

## 🔧 PHẦN 4: Tiền Xử Lý Dữ Liệu

Chuẩn bị dữ liệu cho training CRNN:
- Resize ảnh về chiều cao cố định (32px hoặc 64px)
- Normalize pixel values
- Augmentation: rotation, brightness, noise
- Character encoding (Vietnamese + numbers + punctuation)

In [ ]:
# Cell 9: Xây dựng bộ ký tự (Character Set) cho tiếng Việt
import string

# Bộ ký tự tiếng Việt đầy đủ
vietnamese_chars = (
    "aàảãáạăằẳẵắặâầẩẫấậ"
    "eèẻẽéẹêềểễếệ"
    "iìỉĩíị"
    "oòỏõóọôồổỗốộơờởỡớợ"
    "uùủũúụưừửữứự"
    "yỳỷỹýỵ"
    "AÀẢÃÁẠĂẰẲẴẮẶÂẦẨẪẤẬ"
    "EÈẺẼÉẸÊỀỂỄẾỆ"
    "IÌỈĨÍỊ"
    "OÒỎÕÓỌÔỒỔỖỐỘƠỜỞỠỚỢ"
    "UÙỦŨÚỤƯỪỬỮỨỰ"
    "YỲỶỸÝỴ"
    "dđDĐ"
)

# Số và ký tự đặc biệt thường gặp trong hóa đơn
numbers = "0123456789"
punctuation = ".,;:!?-/()[]{}\"'@#$%&*+=<>| "

# Tạo character vocabulary
charset = sorted(set(vietnamese_chars + string.ascii_letters + numbers + punctuation))
charset = ['<BLANK>'] + charset  # CTC blank token

char_to_idx = {char: idx for idx, char in enumerate(charset)}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

print(f"📚 Vocabulary size: {len(charset)}")
print(f"🔤 Số ký tự tiếng Việt: {len(vietnamese_chars)}")
print(f"\n🔠 Character set preview:")
print(''.join(charset[:100]) + '...')
print(f"\n✅ Đã xây dựng vocabulary với {len(charset)} ký tự")

In [ ]:
# Cell 10: Preprocessing functions
import cv2
import numpy as np
from PIL import Image
import torchvision.transforms as transforms

class OCRPreprocessor:
    """Tiền xử lý ảnh cho CRNN OCR"""
    
    def __init__(self, img_height=64, img_width=None):
        self.img_height = img_height
        self.img_width = img_width  # None = giữ aspect ratio
        
    def resize_with_aspect_ratio(self, img):
        """Resize giữ tỷ lệ, pad nếu cần"""
        h, w = img.shape[:2]
        
        # Scale theo chiều cao
        scale = self.img_height / h
        new_w = int(w * scale)
        new_h = self.img_height
        
        # Resize
        img_resized = cv2.resize(img, (new_w, new_h))
        
        # Nếu có width cố định, pad hoặc crop
        if self.img_width:
            if new_w < self.img_width:
                # Pad bên phải
                pad_width = self.img_width - new_w
                img_resized = np.pad(img_resized, ((0, 0), (0, pad_width), (0, 0)), 
                                    mode='constant', constant_values=255)
            elif new_w > self.img_width:
                # Crop
                img_resized = img_resized[:, :self.img_width, :]
        
        return img_resized
    
    def normalize(self, img):
        """Normalize về [0, 1]"""
        return img.astype(np.float32) / 255.0
    
    def augment(self, img):
        """Data augmentation cho training"""
        # Random rotation ±3 degrees
        if random.random() < 0.3:
            angle = random.uniform(-3, 3)
            h, w = img.shape[:2]
            matrix = cv2.getRotationMatrix2D((w/2, h/2), angle, 1)
            img = cv2.warpAffine(img, matrix, (w, h), 
                                borderMode=cv2.BORDER_CONSTANT, 
                                borderValue=(255, 255, 255))
        
        # Random brightness
        if random.random() < 0.3:
            factor = random.uniform(0.8, 1.2)
            img = np.clip(img * factor, 0, 255).astype(np.uint8)
        
        # Random noise
        if random.random() < 0.2:
            noise = np.random.normal(0, 5, img.shape)
            img = np.clip(img + noise, 0, 255).astype(np.uint8)
        
        return img
    
    def __call__(self, img, augment=False):
        """Full pipeline"""
        # Convert to numpy if PIL Image
        if isinstance(img, Image.Image):
            img = np.array(img)
        
        # Convert grayscale to RGB if needed
        if len(img.shape) == 2:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        
        # Augmentation
        if augment:
            img = self.augment(img)
        
        # Resize
        img = self.resize_with_aspect_ratio(img)
        
        # Normalize
        img = self.normalize(img)
        
        # Convert to torch tensor (C, H, W)
        img = torch.from_numpy(img).permute(2, 0, 1).float()
        
        return img

# Test preprocessor
preprocessor = OCRPreprocessor(img_height=64, img_width=None)
print("✅ OCRPreprocessor đã sẵn sàng!")
print(f"   - Image height: {preprocessor.img_height}px")
print(f"   - Keep aspect ratio: {preprocessor.img_width is None}")

In [ ]:
# Cell 11: Custom Dataset Class
from torch.utils.data import Dataset
import glob

class VietnameseReceiptDataset(Dataset):
    """Dataset cho hóa đơn Việt Nam với annotation JSON"""
    
    def __init__(self, data_dir, preprocessor, char_to_idx, augment=False, max_samples=None):
        self.data_dir = Path(data_dir)
        self.preprocessor = preprocessor
        self.char_to_idx = char_to_idx
        self.augment = augment
        
        # Tìm tất cả ảnh
        self.image_paths = []
        for ext in ['jpg', 'jpeg', 'png', 'JPG', 'JPEG', 'PNG']:
            self.image_paths.extend(glob.glob(str(self.data_dir / f'**/*.{ext}'), recursive=True))
        
        # Filter những ảnh có annotation
        self.samples = []
        for img_path in self.image_paths[:max_samples] if max_samples else self.image_paths:
            json_path = Path(img_path).with_suffix('.json')
            if json_path.exists():
                self.samples.append((img_path, json_path))
        
        print(f"✅ Loaded {len(self.samples)} samples from {data_dir}")
    
    def __len__(self):
        return len(self.samples)
    
    def encode_text(self, text):
        """Encode text thành list of indices"""
        encoded = []
        for char in text:
            if char in self.char_to_idx:
                encoded.append(self.char_to_idx[char])
            # Skip unknown characters
        return encoded
    
    def __getitem__(self, idx):
        img_path, json_path = self.samples[idx]
        
        # Load image
        img = Image.open(img_path).convert('RGB')
        img = self.preprocessor(img, augment=self.augment)
        
        # Load annotation
        with open(json_path, 'r', encoding='utf-8') as f:
            ann = json.load(f)
        
        # Extract text (tùy format của dataset)
        # MC-OCR thường có 'anno_texts' hoặc 'img_info' -> 'text'
        text = ''
        if 'anno_texts' in ann:
            text = ann['anno_texts']
        elif 'img_info' in ann and 'text' in ann['img_info']:
            text = ann['img_info']['text']
        elif 'text' in ann:
            text = ann['text']
        else:
            # Fallback: tìm key chứa text
            for key in ann.keys():
                if isinstance(ann[key], str) and len(ann[key]) > 0:
                    text = ann[key]
                    break
        
        # Encode text
        text_encoded = self.encode_text(text)
        text_length = len(text_encoded)
        
        return img, torch.LongTensor(text_encoded), text_length

# Test dataset
print("🔄 Tạo dataset...")
# Sẽ cập nhật data_dir sau khi khám phá dataset structure

In [ ]:
# Cell 16: Create DataLoaders với cấu trúc dataset thực tế MC-OCR 2021
import pandas as pd
from torch.nn.utils.rnn import pad_sequence

# Cấu trúc dataset MC-OCR 2021 thực tế:
# data/
#   ├── mcocr_train_df.csv
#   ├── mcocr_val_sample_df.csv
#   ├── train_images/train_images/  ← Nested folder!
#   └── val_images/val_images/      ← Nested folder!

# Đọc CSV annotations
train_csv_path = '/content/data/mcocr_train_df.csv'
val_csv_path = '/content/data/mcocr_val_sample_df.csv'

print("📊 Đọc annotation files...")
print(f"Train CSV: {train_csv_path}")
print(f"Val CSV: {val_csv_path}")

train_df = pd.read_csv(train_csv_path)
val_df = pd.read_csv(val_csv_path)

print(f"✅ Train samples: {len(train_df)}")
print(f"✅ Val samples: {len(val_df)}")
print(f"\n📋 Train DataFrame columns: {train_df.columns.tolist()}")
print(f"\n🔍 Sample train annotation:")
print(train_df.head(2))

# Collate function (cần thiết cho DataLoader) - FIX: Pad images to same width
def collate_fn(batch):
    """Collate function để xử lý batch với text và images có độ dài khác nhau"""
    images, texts, text_lengths = zip(*batch)
    
    # Tìm max width trong batch
    max_width = max(img.shape[2] for img in images)
    
    # Pad tất cả images về cùng width
    padded_images = []
    for img in images:
        # img shape: (C, H, W)
        c, h, w = img.shape
        if w < max_width:
            # Pad bên phải với giá trị 0 (hoặc 1 nếu normalized về [0,1])
            pad_width = max_width - w
            # Padding format: (left, right, top, bottom, front, back)
            padded_img = torch.nn.functional.pad(img, (0, pad_width, 0, 0), value=0)
            padded_images.append(padded_img)
        else:
            padded_images.append(img)
    
    # Stack images
    images = torch.stack(padded_images, dim=0)
    
    # Pad texts
    texts_padded = pad_sequence(texts, batch_first=True, padding_value=0)
    
    # Convert to tensors
    text_lengths = torch.LongTensor(text_lengths)
    
    return images, texts_padded, text_lengths

# Tạo custom Dataset cho MC-OCR format
class MCOCRDataset(Dataset):
    """Dataset cho MC-OCR 2021 với CSV annotations"""
    
    def __init__(self, df, img_dir, preprocessor, char_to_idx, augment=False):
        self.df = df.reset_index(drop=True)
        self.img_dir = Path(img_dir)
        self.preprocessor = preprocessor
        self.char_to_idx = char_to_idx
        self.augment = augment
        
        # Verify samples exist
        valid_samples = []
        print(f"\n🔍 Verifying image files in: {self.img_dir}")
        
        for idx in range(len(self.df)):
            row = self.df.iloc[idx]
            # CSV có thể có cột 'img_id', 'file_name', 'image_name', hoặc 'img'
            img_name = row.get('img_id', row.get('file_name', row.get('image_name', row.get('img', None))))
            
            if img_name:
                img_path = self.img_dir / img_name
                if img_path.exists():
                    valid_samples.append(idx)
                elif idx < 5:  # Debug first 5 missing files
                    print(f"   ⚠️ Not found: {img_path}")
        
        self.valid_indices = valid_samples
        print(f"✅ Valid samples: {len(self.valid_indices)} / {len(self.df)}")
        
        if len(self.valid_indices) == 0:
            print("❌ ERROR: No valid samples found!")
            print(f"   Check if images exist in: {self.img_dir}")
            print(f"   Expected image column: {self.df.columns.tolist()}")
    
    def __len__(self):
        return len(self.valid_indices)
    
    def encode_text(self, text):
        """Encode text thành list of indices"""
        if pd.isna(text):
            return []
        encoded = []
        for char in str(text):
            if char in self.char_to_idx:
                encoded.append(self.char_to_idx[char])
        return encoded
    
    def __getitem__(self, idx):
        real_idx = self.valid_indices[idx]
        row = self.df.iloc[real_idx]
        
        # Get image name
        img_name = row.get('img_id', row.get('file_name', row.get('image_name', row.get('img', ''))))
        img_path = self.img_dir / img_name
        
        # Load image
        try:
            img = Image.open(img_path).convert('RGB')
            img = self.preprocessor(img, augment=self.augment)
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            # Return dummy data
            img = torch.zeros(3, 64, 256)
        
        # Get text annotation (có thể là 'anno_texts', 'text', 'label', 'annotation', etc.)
        text = row.get('anno_texts', row.get('text', row.get('label', row.get('annotation', ''))))
        
        # Encode text
        text_encoded = self.encode_text(text)
        if len(text_encoded) == 0:
            text_encoded = [0]  # Dummy để tránh empty tensor
        text_length = len(text_encoded)
        
        return img, torch.LongTensor(text_encoded), text_length

# Tạo datasets với đường dẫn NESTED (train_images/train_images/)
print("\n🔄 Creating datasets...")
train_dataset = MCOCRDataset(
    df=train_df,
    img_dir='/content/data/train_images/train_images',  # Nested folder!
    preprocessor=preprocessor,
    char_to_idx=char_to_idx,
    augment=True
)

val_dataset = MCOCRDataset(
    df=val_df,
    img_dir='/content/data/val_images/val_images',  # Nested folder!
    preprocessor=preprocessor,
    char_to_idx=char_to_idx,
    augment=False
)

# Tạo dataloaders
print("\n🔄 Creating dataloaders...")
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    collate_fn=collate_fn,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    collate_fn=collate_fn,
    pin_memory=True
)

print(f"\n✅ DataLoaders created!")
print(f"   Train batches: {len(train_loader)}")
print(f"   Val batches: {len(val_loader)}")

# Test một batch
if len(train_loader) > 0:
    print("\n📦 Testing sample batch...")
    sample_batch = next(iter(train_loader))
    images, texts, text_lengths = sample_batch
    print(f"   Images shape: {images.shape}")
    print(f"   Texts shape: {texts.shape}")
    print(f"   Text lengths: {text_lengths[:5].tolist()}")
    
    # Decode sample text
    sample_text_indices = texts[0][:min(10, len(texts[0]))].tolist()
    sample_text_chars = [idx_to_char.get(idx, '?') for idx in sample_text_indices]
    print(f"   Sample text (first 10 chars): {''.join(sample_text_chars)}")
    
    print("\n✅ Dataset preparation complete!")
else:
    print("\n❌ ERROR: No data loaded! Check paths and CSV structure.")

## 🏗️ PHẦN 5: Xây Dựng Mô Hình CRNN

**CRNN Architecture:**
1. **CNN Backbone:** Extract visual features (ResNet hoặc VGG)
2. **RNN Layers:** Sequence modeling (LSTM/GRU)
3. **CTC Decoder:** Sequence-to-text decoding

Mô hình này phù hợp cho OCR vì:
- Không cần alignment giữa ảnh và text
- Xử lý được text có độ dài thay đổi
- CTC Loss tự động học alignment

In [ ]:
# Cell 12: CRNN Model Implementation
import torch.nn as nn

class CRNN(nn.Module):
    """CRNN model for Vietnamese Receipt OCR"""
    
    def __init__(self, img_height, num_chars, hidden_size=256, num_rnn_layers=2):
        super(CRNN, self).__init__()
        
        self.img_height = img_height
        self.num_chars = num_chars
        
        # CNN Layers: Extract visual features
        # Input: (batch, 3, H, W) → Output: (batch, 512, H/16, W/4)
        self.cnn = nn.Sequential(
            # Conv Block 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),  # (3, H, W) → (64, H, W)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),  # → (64, H/2, W/2)
            
            # Conv Block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),  # → (128, H/4, W/4)
            
            # Conv Block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d((2, 1), (2, 1)),  # → (256, H/8, W/4)
            
            # Conv Block 4
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d((2, 1), (2, 1)),  # → (512, H/16, W/4)
            
            # Conv Block 5
            nn.Conv2d(512, 512, kernel_size=2, padding=0),  # → (512, H/16-1, W/4-1)
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
        )
        
        # Adaptive pooling để flatten height dimension
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, None))  # → (512, 1, W')
        
        # RNN Layers: Sequence modeling
        self.rnn = nn.LSTM(512, hidden_size, num_rnn_layers, 
                          bidirectional=True, batch_first=True)
        
        # Output Layer: Character prediction
        self.fc = nn.Linear(hidden_size * 2, num_chars)  # *2 for bidirectional
    
    def forward(self, x):
        # CNN feature extraction
        conv_features = self.cnn(x)  # (batch, 512, H', W')
        
        # Adaptive pooling
        conv_features = self.adaptive_pool(conv_features)  # (batch, 512, 1, W')
        conv_features = conv_features.squeeze(2)  # (batch, 512, W')
        
        # Permute for RNN: (batch, W', 512)
        conv_features = conv_features.permute(0, 2, 1)
        
        # RNN sequence modeling
        rnn_output, _ = self.rnn(conv_features)  # (batch, W', hidden_size*2)
        
        # Character prediction
        output = self.fc(rnn_output)  # (batch, W', num_chars)
        
        # Permute for CTC: (W', batch, num_chars)
        output = output.permute(1, 0, 2)
        
        return output

# Test model
model = CRNN(img_height=64, num_chars=len(charset), hidden_size=256, num_rnn_layers=2)
model = model.to(device)

# Test forward pass
dummy_input = torch.randn(2, 3, 64, 256).to(device)
dummy_output = model(dummy_input)
print(f"✅ CRNN Model initialized!")
print(f"   Input shape: {dummy_input.shape}")
print(f"   Output shape: {dummy_output.shape}")  # (seq_len, batch, num_chars)
print(f"   Parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

In [ ]:
# Cell 13: Collate function cho DataLoader (xử lý variable-length sequences)
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    """
    Collate function để xử lý batch với text có độ dài khác nhau
    """
    images, texts, text_lengths = zip(*batch)
    
    # Stack images
    images = torch.stack(images, dim=0)
    
    # Pad texts
    texts_padded = pad_sequence(texts, batch_first=True, padding_value=0)
    
    # Convert to tensors
    text_lengths = torch.LongTensor(text_lengths)
    
    return images, texts_padded, text_lengths

print("✅ Collate function ready!")

## 🔥 PHẦN 6: Training Setup & Loop

Cấu hình training:
- **Optimizer:** Adam với learning rate 0.0005
- **Scheduler:** ReduceLROnPlateau
- **Loss:** CTCLoss (Connectionist Temporal Classification)
- **Metrics:** CER (Character Error Rate)
- **Epochs:** 50-100 epochs
- **Batch Size:** 32 (tùy GPU memory)

In [ ]:
# Cell 14: Training configuration & helper functions
import editdistance

# Hyperparameters
BATCH_SIZE = 32
LEARNING_RATE = 0.0005
NUM_EPOCHS = 50
SAVE_DIR = '/content/checkpoints'
os.makedirs(SAVE_DIR, exist_ok=True)

# Optimizer & Scheduler
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

# CTC Loss
ctc_loss = nn.CTCLoss(blank=0, zero_infinity=True)

# Decoder function
def decode_predictions(preds, idx_to_char):
    """Decode CTC output to text"""
    # preds shape: (seq_len, batch, num_chars)
    # Get argmax
    _, max_indices = torch.max(preds, dim=2)  # (seq_len, batch)
    max_indices = max_indices.transpose(0, 1).cpu().numpy()  # (batch, seq_len)
    
    decoded_texts = []
    for indices in max_indices:
        # Remove blanks and duplicates (CTC decoding)
        chars = []
        prev_idx = None
        for idx in indices:
            if idx != 0 and idx != prev_idx:  # 0 is blank
                if idx < len(idx_to_char):
                    chars.append(idx_to_char[idx])
            prev_idx = idx
        decoded_texts.append(''.join(chars))
    
    return decoded_texts

# Character Error Rate
def calculate_cer(predictions, targets):
    """Calculate Character Error Rate"""
    total_distance = 0
    total_length = 0
    
    for pred, target in zip(predictions, targets):
        distance = editdistance.eval(pred, target)
        total_distance += distance
        total_length += len(target)
    
    cer = total_distance / total_length if total_length > 0 else 0
    return cer

print("✅ Training configuration ready!")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Learning rate: {LEARNING_RATE}")
print(f"   Epochs: {NUM_EPOCHS}")

## ⚠️ QUAN TRỌNG: Đọc File OCR_TRAINING_GUIDE.md

Do giới hạn của notebook format, tôi đã tạo file **`OCR_TRAINING_GUIDE.md`** chứa:

✅ **Full training loop code** (copy-paste vào cells bên dưới)  
✅ **DataLoader creation code**  
✅ **Inference & demo code**  
✅ **ONNX export code**  
✅ **TFLite conversion code**  
✅ **Flutter integration guide**  
✅ **Troubleshooting tips**

**Workflow:**
1. Chạy cells 1-14 trong notebook này
2. Mở file `OCR_TRAINING_GUIDE.md`
3. Copy code từ sections tương ứng
4. Paste vào các cells bên dưới
5. Chạy training!

---

### 🔥 Quick Start:

Các bước tiếp theo (copy từ OCR_TRAINING_GUIDE.md):

**Cell 15:** Training loop functions  
**Cell 16:** Create DataLoaders  
**Cell 17:** Start training  
**Cell 18-19:** Inference & demo  
**Cell 20:** Export ONNX  
**Cell 21:** Export TFLite  
**Cell 22:** Download models

In [ ]:
# Cell 15: Placeholder - Copy training loop từ OCR_TRAINING_GUIDE.md
# Copy toàn bộ section "1️⃣ COMPLETE TRAINING LOOP CODE" vào đây

print("⚠️ Chưa có training loop!")
print("📖 Mở file OCR_TRAINING_GUIDE.md")
print("📋 Copy section 1️⃣ vào cell này")
print("▶️ Sau đó chạy cell")

In [ ]:
# Cell 16: Placeholder - Create DataLoaders
# Copy section "2️⃣ CREATE DATALOADERS" từ OCR_TRAINING_GUIDE.md

print("⚠️ Cell cần được điền code từ OCR_TRAINING_GUIDE.md")
print("📋 Section: 2️⃣ CREATE DATALOADERS")

In [ ]:
# Cell 17: Placeholder - Execute Training
# Copy section "3️⃣ START TRAINING" từ OCR_TRAINING_GUIDE.md

print("⚠️ Cell cần được điền code từ OCR_TRAINING_GUIDE.md")
print("📋 Section: 3️⃣ START TRAINING")

## 🎯 PHẦN 7: Inference & Model Export

Sau khi training xong:
1. Test model với ảnh mẫu
2. Export sang ONNX
3. Convert sang TFLite
4. Download models về máy

Copy code từ **OCR_TRAINING_GUIDE.md** sections 4️⃣-7️⃣

In [ ]:
# Cell 18-22: Placeholders cho Inference & Export
# Copy từ OCR_TRAINING_GUIDE.md:
# - Section 4️⃣: INFERENCE & DEMO
# - Section 5️⃣: EXPORT TO ONNX
# - Section 6️⃣: EXPORT TO TFLITE
# - Section 7️⃣: DOWNLOAD MODELS

print("📖 Mở OCR_TRAINING_GUIDE.md để copy code")
print("✅ Hoàn thành các cells 1-14")
print("📋 Copy sections 4️⃣-7️⃣ vào cells tiếp theo")
print("\n🎯 Kết quả cuối cùng:")
print("   - best_model.pth (PyTorch checkpoint)")
print("   - vietnamese_ocr_model.onnx (ONNX format)")
print("   - vietnamese_ocr_model.tflite (TFLite for Flutter)")
print("   - training_curves.png (Visualization)")

### 💡 Hướng dẫn sử dụng Cell Test OCR:

**Cách 1: Upload ảnh của bạn**
1. Chạy cell trên
2. Click "Choose Files" khi được prompt
3. Chọn ảnh hóa đơn từ máy tính
4. Xem kết quả OCR

**Cách 2: Test với ảnh từ dataset**
- Cell sẽ tự động lấy 1 ảnh random từ validation set
- So sánh Ground Truth vs Prediction
- Tính CER (Character Error Rate)

**Kết quả hiển thị:**
- ✅ Ảnh gốc
- ✅ Text nhận dạng được
- ✅ So sánh với ground truth (nếu có)
- ✅ CER score

---

### 📊 Cell 24: Batch Test Features

**Tính năng:**
- ✅ Test nhiều ảnh cùng lúc (default: 10 ảnh)
- ✅ Tính CER cho từng ảnh
- ✅ Thống kê tổng hợp: Average, Best, Worst CER
- ✅ Phân loại performance: Excellent/Good/Fair/Poor
- ✅ Visualization: So sánh ảnh tốt nhất vs tệ nhất
- ✅ Export kết quả: `batch_test_results.png`

**Điều chỉnh số lượng:**
```python
# Test với 20 ảnh thay vì 10
batch_results = batch_test_ocr(num_samples=20, ...)
```

---

In [ ]:
# Cell 24: Test OCR với nhiều ảnh (Batch Test)
import torch
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

def batch_test_ocr(num_samples=10, model=None, idx_to_char=None, dataset=None, device='cuda'):
    """
    Test OCR với nhiều ảnh từ validation set
    """
    if model is None or idx_to_char is None or dataset is None:
        print("❌ Cần load model và dataset trước!")
        return None
    
    print(f"\n{'='*80}")
    print(f"🧪 Batch OCR Test - Testing {num_samples} random images")
    print(f"{'='*80}\n")
    
    # Random sampling
    import random
    sample_indices = random.sample(range(len(dataset.valid_indices)), min(num_samples, len(dataset.valid_indices)))
    
    results = []
    total_cer = 0
    
    for i, val_idx in enumerate(tqdm(sample_indices, desc="Testing")):
        real_idx = dataset.valid_indices[val_idx]
        row = dataset.df.iloc[real_idx]
        
        img_path = f"/content/data/train_images/train_images/{row['img_id']}"
        ground_truth = row['anno_texts']
        
        # Preprocess
        img_tensor, _ = preprocess_test_image(img_path)
        if img_tensor is None:
            continue
        
        img_tensor = img_tensor.to(device)
        
        # Inference
        model.eval()
        with torch.no_grad():
            logits = model(img_tensor)
        
        # Decode
        predicted = decode_ctc_predictions(logits, idx_to_char)
        
        # Calculate CER
        def calculate_cer(gt, pred):
            import difflib
            s = difflib.SequenceMatcher(None, gt, pred)
            total = len(gt)
            errors = total - sum(block.size for block in s.get_matching_blocks())
            return (errors / total * 100) if total > 0 else 0
        
        cer = calculate_cer(ground_truth, predicted)
        total_cer += cer
        
        results.append({
            'image': row['img_id'],
            'ground_truth': ground_truth,
            'predicted': predicted,
            'cer': cer
        })
    
    if not results:
        print("❌ Không có kết quả nào!")
        return None
    
    # Display results
    print(f"\n{'='*80}")
    print("📊 BATCH TEST RESULTS")
    print(f"{'='*80}\n")
    
    for i, result in enumerate(results, 1):
        print(f"🔹 Image {i}: {result['image']}")
        print(f"   Ground Truth: {result['ground_truth'][:50]}{'...' if len(result['ground_truth']) > 50 else ''}")
        print(f"   Predicted:    {result['predicted'][:50]}{'...' if len(result['predicted']) > 50 else ''}")
        print(f"   CER: {result['cer']:.2f}%")
        
        # Status indicator
        if result['cer'] < 10:
            status = "✅ Excellent"
        elif result['cer'] < 20:
            status = "👍 Good"
        elif result['cer'] < 30:
            status = "⚠️  Fair"
        else:
            status = "❌ Poor"
        print(f"   {status}")
        print()
    
    # Summary
    avg_cer = total_cer / len(results) if results else 0
    
    print(f"{'='*80}")
    print("📈 SUMMARY STATISTICS")
    print(f"{'='*80}")
    print(f"✅ Total Images Tested: {len(results)}")
    print(f"📊 Average CER: {avg_cer:.2f}%")
    print(f"🏆 Best CER: {min(r['cer'] for r in results):.2f}%")
    print(f"📉 Worst CER: {max(r['cer'] for r in results):.2f}%")
    
    # Performance breakdown
    excellent = sum(1 for r in results if r['cer'] < 10)
    good = sum(1 for r in results if 10 <= r['cer'] < 20)
    fair = sum(1 for r in results if 20 <= r['cer'] < 30)
    poor = sum(1 for r in results if r['cer'] >= 30)
    
    print(f"\n📊 Performance Breakdown:")
    print(f"   ✅ Excellent (< 10%):  {excellent} ({excellent/len(results)*100:.1f}%)")
    print(f"   👍 Good (10-20%):      {good} ({good/len(results)*100:.1f}%)")
    print(f"   ⚠️  Fair (20-30%):      {fair} ({fair/len(results)*100:.1f}%)")
    print(f"   ❌ Poor (> 30%):       {poor} ({poor/len(results)*100:.1f}%)")
    
    print(f"\n{'='*80}\n")
    
    # Visualize best and worst
    try:
        print("🖼️  Visualizing best and worst predictions...")
        
        sorted_results = sorted(results, key=lambda x: x['cer'])
        best_result = sorted_results[0]
        worst_result = sorted_results[-1]
        
        fig, axes = plt.subplots(2, 2, figsize=(16, 8))
        
        for idx, (result, title_prefix) in enumerate([(best_result, 'BEST'), (worst_result, 'WORST')]):
            img_path = f"/content/data/train_images/train_images/{result['image']}"
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            axes[idx, 0].imshow(img)
            axes[idx, 0].set_title(f"{title_prefix} - {result['image']} (CER: {result['cer']:.2f}%)", 
                                   fontweight='bold', fontsize=12)
            axes[idx, 0].axis('off')
            
            text_display = f"Ground Truth:\n{result['ground_truth'][:100]}\n\n"
            text_display += f"Predicted:\n{result['predicted'][:100]}\n\n"
            text_display += f"CER: {result['cer']:.2f}%"
            
            axes[idx, 1].text(0.1, 0.5, text_display, 
                             ha='left', va='center', 
                             fontsize=10, wrap=True,
                             bbox=dict(boxstyle='round', 
                                      facecolor='lightgreen' if result['cer'] < 20 else 'lightcoral',
                                      alpha=0.7))
            axes[idx, 1].set_xlim(0, 1)
            axes[idx, 1].set_ylim(0, 1)
            axes[idx, 1].axis('off')
            axes[idx, 1].set_title("OCR Result", fontweight='bold', fontsize=12)
        
        plt.tight_layout()
        plt.savefig('batch_test_results.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        print("\n✅ Visualization saved: batch_test_results.png")
    except Exception as e:
        print(f"⚠️  Could not visualize: {e}")
    
    return results

# ═══════════════════════════════════════════════════════════════════════
# 🎯 TỰ ĐỘNG CHẠY BATCH TEST:
# ═══════════════════════════════════════════════════════════════════════

print("\n📊 Batch Test Cell")
print("="*70)

try:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Check prerequisites
    if 'model' not in dir():
        raise NameError("model")
    if 'idx_to_char' not in dir():
        raise NameError("idx_to_char")
    if 'val_dataset' not in dir():
        raise NameError("val_dataset")
    if 'preprocess_test_image' not in dir():
        raise NameError("preprocess_test_image (chạy Cell 23 trước)")
    if 'decode_ctc_predictions' not in dir():
        raise NameError("decode_ctc_predictions (chạy Cell 23 trước)")
    
    print("✅ Tất cả prerequisites đã sẵn sàng!")
    print(f"✅ Device: {device}")
    
    # Chạy batch test với 10 ảnh
    print("\n🚀 Đang chạy batch test với 10 ảnh ngẫu nhiên...")
    batch_results = batch_test_ocr(
        num_samples=10,
        model=model,
        idx_to_char=idx_to_char,
        dataset=val_dataset,
        device=device
    )
    
    print("\n✅ Batch test hoàn tất!")
    print("="*70)
    
    print("\n💡 Để test nhiều ảnh hơn:")
    print("   batch_results = batch_test_ocr(")
    print("       num_samples=20,  # Thay đổi số lượng")
    print("       model=model,")
    print("       idx_to_char=idx_to_char,")
    print("       dataset=val_dataset,")
    print("       device=device")
    print("   )")

except NameError as e:
    print(f"❌ Thiếu: {e}")
    print("\n💡 Cần chạy trước:")
    print("   - Cell 9: charset & idx_to_char")
    print("   - Cell 12: model CRNN")
    print("   - Cell 16: val_dataset")
    print("   - Cell 23: preprocess_test_image & decode_ctc_predictions")
    print("   - Cells 15-17: training")
    print()
    print("📋 Sau khi train xong, chạy Cell 23 trước rồi mới chạy Cell 24 này")


In [ ]:
# Cell 23: Test OCR với ảnh hóa đơn bất kỳ
import torch
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def preprocess_test_image(image_path, target_height=64):
    """
    Tiền xử lý ảnh giống như trong training
    """
    # Đọc ảnh
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Không thể đọc ảnh: {image_path}")
        return None, None
    
    # Convert BGR to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Resize về height=64, giữ aspect ratio
    h, w = img.shape[:2]
    new_h = target_height
    new_w = int(w * (new_h / h))
    img_resized = cv2.resize(img, (new_w, new_h))
    
    # Normalize [0, 255] -> [0, 1]
    img_normalized = img_resized.astype(np.float32) / 255.0
    
    # Convert to tensor [C, H, W]
    img_tensor = torch.from_numpy(img_normalized).permute(2, 0, 1)
    
    # Add batch dimension [1, C, H, W]
    img_tensor = img_tensor.unsqueeze(0)
    
    return img_tensor, img_resized

def decode_ctc_predictions(logits, idx_to_char):
    """
    Giải mã CTC predictions thành text
    """
    # logits shape: [seq_len, batch_size=1, num_chars]
    _, preds = logits.max(2)  # [seq_len, 1]
    preds = preds.squeeze(1)  # [seq_len]
    
    # CTC decoding: remove blanks and duplicates
    decoded_chars = []
    prev_char = None
    
    for idx in preds:
        idx = idx.item()
        if idx == 0:  # blank token
            prev_char = None
            continue
        if idx == prev_char:  # duplicate
            continue
        if idx in idx_to_char:
            decoded_chars.append(idx_to_char[idx])
        prev_char = idx
    
    return ''.join(decoded_chars)

def test_image_ocr(image_path, model, idx_to_char, device='cuda'):
    """
    Test OCR với 1 ảnh hóa đơn
    """
    print(f"\n{'='*70}")
    print(f"🧪 Testing OCR with image: {image_path}")
    print(f"{'='*70}")
    
    # Tiền xử lý
    img_tensor, img_display = preprocess_test_image(image_path)
    if img_tensor is None:
        return None
    
    # Move to device
    img_tensor = img_tensor.to(device)
    
    # Inference
    model.eval()
    with torch.no_grad():
        logits = model(img_tensor)  # [seq_len, 1, num_chars]
    
    # Decode
    predicted_text = decode_ctc_predictions(logits, idx_to_char)
    
    # Display results
    plt.figure(figsize=(15, 5))
    
    # Show image
    plt.subplot(1, 2, 1)
    plt.imshow(img_display)
    plt.title('Input Image', fontsize=14, fontweight='bold')
    plt.axis('off')
    
    # Show prediction
    plt.subplot(1, 2, 2)
    plt.text(0.5, 0.5, predicted_text, 
             ha='center', va='center', 
             fontsize=12, wrap=True,
             bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.axis('off')
    plt.title('OCR Prediction', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Print results
    print(f"\n📝 Recognized Text:")
    print(f"{'─'*70}")
    print(f"{predicted_text}")
    print(f"{'─'*70}")
    print(f"\n✅ Text Length: {len(predicted_text)} characters")
    print(f"✅ Image Size: {img_display.shape[1]}x{img_display.shape[0]}px")
    print(f"✅ Tensor Shape: {img_tensor.shape}")
    
    return predicted_text

# ═══════════════════════════════════════════════════════════════════════
# 🎯 TỰ ĐỘNG CHẠY TEST:
# ═══════════════════════════════════════════════════════════════════════

print("\n📋 OCR Test Cell")
print("="*70)

try:
    # Check device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"✅ Device: {device}")
    
    # Check if model exists
    if 'model' not in dir():
        raise NameError("Model chưa được define")
    
    if 'idx_to_char' not in dir():
        raise NameError("idx_to_char chưa được define")
    
    print("✅ Model và idx_to_char đã sẵn sàng!")
    
    # Kiểm tra xem có val_dataset không
    if 'val_dataset' in dir():
        # Option 1: Test với ảnh từ validation set
        print("\n🔄 Đang test với ảnh ngẫu nhiên từ validation set...")
        
        import random
        val_idx = random.randint(0, len(val_dataset.valid_indices) - 1)
        real_idx = val_dataset.valid_indices[val_idx]
        sample_row = val_dataset.df.iloc[real_idx]
        
        sample_img_path = f"/content/data/train_images/train_images/{sample_row['img_id']}"
        sample_text = sample_row['anno_texts']
        
        print(f"\n📌 Sample from validation set:")
        print(f"   - Image: {sample_row['img_id']}")
        print(f"   - Ground Truth: {sample_text[:100]}{'...' if len(sample_text) > 100 else ''}")
        
        # Test OCR
        predicted = test_image_ocr(sample_img_path, model, idx_to_char, device)
        
        # Compare with ground truth
        print(f"\n📊 COMPARISON:")
        print(f"{'='*70}")
        print(f"Ground Truth:\n{sample_text}")
        print(f"\n{'-'*70}")
        print(f"Predicted:\n{predicted}")
        print(f"{'='*70}")
        
        # Calculate CER
        def calculate_cer(ground_truth, predicted):
            import difflib
            s = difflib.SequenceMatcher(None, ground_truth, predicted)
            total_chars = len(ground_truth)
            errors = total_chars - sum(block.size for block in s.get_matching_blocks())
            cer = errors / total_chars if total_chars > 0 else 0
            return cer * 100
        
        cer = calculate_cer(sample_text, predicted)
        print(f"\n📈 Character Error Rate (CER): {cer:.2f}%")
        
        if cer < 10:
            print("   ✅ Excellent! (< 10%)")
        elif cer < 20:
            print("   👍 Good! (10-20%)")
        elif cer < 30:
            print("   ⚠️  Fair (20-30%)")
        else:
            print("   ❌ Needs improvement (> 30%)")
    
    else:
        # Option 2: Upload ảnh từ máy tính
        print("\n⚠️  Không tìm thấy val_dataset")
        print("📤 Vui lòng upload ảnh hóa đơn để test OCR:\n")
        
        from google.colab import files
        uploaded = files.upload()
        
        if uploaded:
            test_image_path = list(uploaded.keys())[0]
            print(f"\n✅ Uploaded: {test_image_path}")
            
            # Test OCR
            predicted = test_image_ocr(test_image_path, model, idx_to_char, device)
            
            print(f"\n📝 Kết quả OCR:")
            print(f"{'='*70}")
            print(predicted)
            print(f"{'='*70}")
        else:
            print("❌ Không có file nào được upload")
    
    print("\n" + "="*70)
    print("✅ Test hoàn tất!")
    print("="*70)
    
    print("\n💡 Để test thêm ảnh:")
    print("   from google.colab import files")
    print("   uploaded = files.upload()")
    print("   test_image_path = list(uploaded.keys())[0]")
    print("   result = test_image_ocr(test_image_path, model, idx_to_char, device)")

except NameError as e:
    print(f"❌ Error: {e}")
    print("\n💡 Cell này cần các biến sau từ training:")
    print("   - model (từ Cell 12 và training)")
    print("   - idx_to_char (từ Cell 9)")
    print()
    print("📋 Đảm bảo đã chạy:")
    print("   1. Cells 1-14 (setup & config)")
    print("   2. Training đã hoàn tất")
    print("   3. Model đã được load vào biến 'model'")


## 🧪 PHẦN 8: Test OCR với ảnh bất kỳ

Test model đã train với ảnh hóa đơn bất kỳ để kiểm tra độ chính xác

---

## 📚 TÀI LIỆU THAM KHẢO

### 📖 Files đi kèm:
1. **OCR_TRAINING_GUIDE.md** - Hướng dẫn chi tiết với full code
2. **OCR_TROUBLESHOOTING.md** - Fix lỗi OCR trong app
3. **vietnamese_receipt_ocr_training.ipynb** - Notebook này

### 🔗 Resources:
- [MC-OCR 2021 Dataset](https://www.kaggle.com/datasets/domixi1989/vietnamese-receipts-mc-ocr-2021)
- [CRNN Paper](https://arxiv.org/abs/1507.05717)
- [CTC Loss Explained](https://distill.pub/2017/ctc/)
- [TFLite Flutter Plugin](https://pub.dev/packages/tflite_flutter)

### ⚙️ System Requirements:
- **Google Colab:** Pro recommended (GPU T4/V100)
- **RAM:** 12GB+ for full dataset
- **Storage:** 5GB+ for dataset + models
- **Training Time:** 2-4 hours

### 🎯 Expected Performance:
- **CER:** 5-15% on MC-OCR test set
- **Model Size:** ~25MB (TFLite FP16)
- **Inference Speed:** 200-500ms/image on mobile

---

## ✅ CHECKLIST HOÀN THÀNH

**Setup:**
- [ ] Upload kaggle.json
- [ ] Download dataset thành công
- [ ] Verify dataset structure

**Training:**
- [ ] Chạy cells 1-14 không lỗi
- [ ] Copy code từ OCR_TRAINING_GUIDE.md
- [ ] Training hoàn thành
- [ ] Val CER < 20%

**Export:**
- [ ] ONNX model exported
- [ ] TFLite model exported
- [ ] Models downloaded về máy

**Integration:**
- [ ] Copy TFLite vào Flutter /assets/models/
- [ ] Update pubspec.yaml
- [ ] Test inference trong app

---

**🎉 Chúc bạn training thành công!**

Nếu gặp vấn đề, check:
1. OCR_TRAINING_GUIDE.md - Full code
2. OCR_TROUBLESHOOTING.md - Fix lỗi
3. GitHub Issues - Báo bug

**Author:** KHANH | **Date:** Nov 17, 2025 | **Version:** 1.0